# Finger Prediction Model

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img

from tensorflow.keras.applications import ResNet50

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

gen = ImageDataGenerator()

generator = gen.flow_from_directory("FINGERS DATASET/train")

Found 18000 images belonging to 6 classes.


In [5]:
# Generating the ResNet50 Model

resnet_model = ResNet50(
    include_top=False,
    weights="imagenet",
    input_shape=(256, 256,3))

94773248/94765736 [==============================] - 1s 0us/step


In [6]:
for layer in resnet_model.layers:
  layer.trainable = False

In [7]:
resnet_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 262, 262, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 128, 128, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 128, 128, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

### Generating Model for Finger Prediction

In [8]:
from tensorflow.keras.layers import Input, Dense, Flatten
from tensorflow.keras.models import Model

In [9]:
in_layer = Input(shape=(256,256,3))
resnet_layer = resnet_model(in_layer)
flat = Flatten()(resnet_layer)
d1 = Dense(units=400, activation="relu")(flat)
d2 = Dense(units=200, activation="relu")(d1)
out_layer = Dense(units=6, activation="softmax")(d2)

In [10]:
model = Model(in_layer, out_layer)

In [11]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
resnet50 (Model)             (None, 8, 8, 2048)        23587712  
_________________________________________________________________
flatten (Flatten)            (None, 131072)            0         
_________________________________________________________________
dense (Dense)                (None, 400)               52429200  
_________________________________________________________________
dense_1 (Dense)              (None, 200)               80200     
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 1206      
Total params: 76,098,318
Trainable params: 52,510,606
Non-trainable params: 23,587,712
________________________________________

In [12]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [13]:
model.fit_generator(generator, steps_per_epoch=20, epochs=5)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/5
20/20 [==============================] - 3s 136ms/step - loss: 13.1533 - accuracy: 0.7234
Epoch 2/5
20/20 [==============================] - 3s 135ms/step - loss: 0.7855 - accuracy: 0.9703
Epoch 3/5
20/20 [==============================] - 3s 136ms/step - loss: 0.0185 - accuracy: 0.9984
Epoch 4/5
20/20 [==============================] - 3s 136ms/step - loss: 3.0054e-05 - accuracy: 1.0000
Epoch 5/5
20/20 [==============================] - 3s 135ms/step - loss: 5.3084e-08 - accuracy: 1.0000


In [15]:
# Saving the model and model weights of the Finger Prediction Model

model.save('Model.h5')
model.save_weights('Model_weights.h5')